In [ ]:
from __future__ import unicode_literals # 이게 뭘까
import requests
import pymongo
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# from scrapy.http import TextResponse
import urllib.request
import urllib
import getpass
import time
import re
# from google.cloud import vision
# from google.cloud.vision import types
import os
import io
from os import walk
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys

In [ ]:
!cd

In [ ]:
if not os.path.exists('./thumb_detail'):
    os.mkdir('./thumb_detail')
    
URL = 'https://page.kakao.com/main?categoryUid=10&subCategoryUid=1002'

driver = webdriver.Chrome(executable_path='C:\chromedriver')
driver.implicitly_wait(time_to_wait=1)
driver.get(URL)
time.sleep(1)

driver.find_element_by_xpath('//*[@id="kpw-header"]/div/div/button/div[3]').click()

driver.switch_to_window(driver.window_handles[1])
driver.get_window_position(driver.window_handles[1])

driver.find_element_by_xpath('//*[@id="id_email_2"]').send_keys('아이디 입력')
driver.find_element_by_xpath('//*[@id="id_password_3"]').send_keys('비밀번호 입력')
driver.find_element_by_xpath('//*[@id="login-form"]/fieldset/div[8]/button[1]').click()
time.sleep(1)

driver.switch_to_window(driver.window_handles[0])
driver.get_window_position(driver.window_handles[0])

#요일별&완결 웹툰 회차별 썸네일 가져오기
for i in range(1,9):
    
    driver.find_element_by_xpath('//*[@id="root"]/div[3]/div[2]/ul/li['+str(i)+']/div').click()
    time.sleep(1)
    
    if i != 8:
        for key_down in range(0,20):
            driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
            time.sleep(0.5)
    else:
        for key_down in range(0,100):
            driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
            time.sleep(0.5)

    domain_url = 'https://page.kakao.com'
    main_url = 'https://page.kakao.com/main?categoryUid=10&subCategoryUid=1002'

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    webtoon_area = soup.select('#root > div.jsx-3157985592.mainContents.mainContents_pc > div.css-1sna24c > div.css-1saqd06 > div > a')
#     webtoon_area = webtoon_area[130:] #수집 도중 오류 발생 시 해당 번호부터 다시 수집
    done_num=1

    for each_webtoon in webtoon_area:
        webtoon_last_page = domain_url + each_webtoon['href'] + '&page={}'.format(999)

        driver.get(webtoon_last_page)
        time.sleep(0.5)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        driver.find_element_by_css_selector('#root > div.jsx-885677927.mainContents.mainContents_pc.hiddenMenuContent > div > div > \
                                    div.css-sgpdfd > div > div.css-1ydjg2i > div.css-1nlm7ol > div.css-82j595 > button.css-zkp4tp').click()
        time.sleep(0.5)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        author_list = soup.find_all('span', 'jsx-3755015728')
        
        writer = ''
        illustrator = ''
        
        for il in range(len(author_list)//2):
            if author_list[2 * il].text == '글':
                writer += author_list[2*il+1].text
            elif author_list[2 * il].text == '그림':
                illustrator += author_list[2*il+1].text
            elif author_list[2 * il].text == '원작':
                origin = author_list[2*il+1].text
                if origin in writer:
                    continue
                elif writer == '':
                    writer = origin
                else:
                    writer += ',' + origin

        if writer == '':
            writer = illustrator
        if illustrator == '':
            illustrator = writer

        if len(author_list) == 1:
            writer = illustrator = author_list[0].text
            
        illustrator = re.sub('[\\/:*?"<>|]', '',illustrator)
        
        if len(illustrator) > 15:
            illustrator = illustrator.split(',')[0]

        print(illustrator)

        driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[1]/i[2]').click()

        try:
            last_page_num = int(soup.select('#root > div.jsx-885677927.mainContents.mainContents_pc.hiddenMenuContent > div > div > div.css-lro00e > div.css-ma6gbc')[0].find_all('div')[-1].text)
            time.sleep(0.5)
        except:
            last_page_num = 1
#         print(last_page_num)

        for page_num in range(1, last_page_num + 1):
            webtoon_url = domain_url + each_webtoon['href'] + '&page={}'.format(page_num)

            driver.get(webtoon_url)
            time.sleep(0.5)
            
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            
            webtoon_name = soup.findAll('div',{'class':'css-4cffwv'})
            webtoon_name = re.sub('[\\/:*?"<>|]', '',webtoon_name[0].text)
            
            author = soup.findAll('div',{'class':'text-ellipsis css-7a7cma'})
            author = re.sub('[\\/:*?"<>|]', '',author[1].text)

            data_list=soup.findAll('ul',{'class':'css-14gr98z'})

            li_list = []
            for data in data_list:
                li_list.extend(data.findAll('li')) #해당 부분을 찾아 li_list와 병합

            thumb = []

            for li in li_list:
                img = li.find('img')
                img_src = img['data-src']
                thumb.append('https:' + img_src)

            titles = soup.findAll('div',{'class':'text-ellipsis css-1mn7vax'})
            title_list = [t.text for t in titles]

            title = []
            for p in range(len(title_list)):
                title.append(re.sub('[\\/:*?"<>|]','',title_list[p]))

            for k in range(len(thumb)):
                if not os.path.exists('./thumb_detail/{}'.format(illustrator)):
                    os.mkdir('./thumb_detail/{}'.format(illustrator))
                    
                urllib.request.urlretrieve(thumb[k], './thumb_detail/{}/{}{}{}.jpg'.format(illustrator,webtoon_name,'_',title[k]))

        print(str(done_num) + ' ' + webtoon_name + ' 완료')
        done_num+=1
                
    driver.get('https://page.kakao.com/main?categoryUid=10&subCategoryUid=1002&day=1')
    print(str(i) + ' finish')

driver.close()
print('크롤링 완료')